In [1]:
%cd ..

/volatile/home/Zaccharie/workspace/understanding-unets


In [2]:
# # this just to make sure we are using only on CPU
# import os
# os.environ["CUDA_VISIBLE_DEVICES"]="-1"

In [3]:
%load_ext autoreload
%autoreload 2
import os.path as op
import time

from keras.callbacks import TensorBoard, ModelCheckpoint
from keras_tqdm import TQDMNotebookCallback
from tensorflow import set_random_seed
from tqdm import tqdm_notebook

from data import keras_im_generator, im_generator_BSD68
from evaluate import psnr, ssim
from unet import unet

Using TensorFlow backend.


In [4]:
set_random_seed(1)

In [5]:
batch_size = 1
noise_std = 30 / 255
source = 'bsd68'
validation_split = 0.1
if 'cifar' in source:
    n_samples_train = 5*1e4
    size = 32
elif 'mnist' in source:
    n_samples_train = 6*1e4
    size = 28
elif source == 'bsd68':
    bsd_dir_train = 'BSDS300/images/train'
    n_samples_train = 200
    size = None
if source in ['cifar', 'cifar_grey', 'mnist']:
    im_gen_train = keras_im_generator(mode='training', validation_split=0.1, batch_size=batch_size, source=source, noise_std=noise_std)
    im_gen_val = keras_im_generator(mode='validation', validation_split=0.1, batch_size=batch_size, source=source, noise_std=noise_std)
elif source == 'bsd68':
    im_gen_train = im_generator_BSD68(path=bsd_dir_train, mode='training', validation_split=0.1, batch_size=batch_size, noise_std=noise_std)
    im_gen_val = im_generator_BSD68(path=bsd_dir_train, mode='validation', validation_split=0.1, batch_size=batch_size, noise_std=noise_std)

In [6]:
run_params = {
#     'n_layers': 4, 
    'n_layers': 2, 
    'pool': 'max', 
#     "layers_n_channels": [16, 32, 64, 128], 
    "layers_n_channels": [16, 32], 
    'layers_n_non_lins': 2,
    'non_relu_contract': False,
}
n_epochs = 50
run_id = f'unet_{source}_{int(noise_std * 255)}_{int(time.time())}'
chkpt_path = f'checkpoints/{run_id}' + '-{epoch:02d}.hdf5'
print(run_id)

unet_bsd68_30_1570190201


In [7]:
chkpt_cback = ModelCheckpoint(chkpt_path, period=1000, save_weights_only=False)
log_dir = op.join('logs', run_id)
tboard_cback = TensorBoard(
    log_dir=log_dir, 
    histogram_freq=0, 
    write_graph=True, 
    write_images=False, 
)
tqdm_cb = TQDMNotebookCallback(metric_format="{name}: {value:e}")

In [8]:
model = unet(input_size=(size, size, 3), lr=1e-3, **run_params)
print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 1 448         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, None, None, 1 2320        conv2d_1[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, None, None, 1 0           conv2d_2[0][0]                   
_____________________________________

In [9]:
# model.fit_generator(
#     im_gen_train, 
#     steps_per_epoch=int((1-validation_split) * n_samples_train / batch_size), 
#     epochs=n_epochs,
#     validation_data=im_gen_val,
#     validation_steps=int(validation_split * n_samples_train / batch_size),
#     verbose=0,
#     callbacks=[tqdm_cb, tboard_cback, chkpt_cback,],
# #     max_queue_size=100,
#     use_multiprocessing=True,
#     workers=35,
# )

In [10]:
%%time
# overfitting trials
data = next(im_gen_train)
val_data = next(im_gen_val)
model.fit(
    x=data[0], 
    y=data[1], 
#     validation_data=val_data, 
    batch_size=data[0].shape[0], 
    epochs=500, 
    verbose=2, 
    shuffle=False,
)
print('Original metrics')
print(psnr(*data))

Instructions for updating:
Use tf.cast instead.
Epoch 1/500
 - 3s - loss: 6.9244 - keras_psnr: 4.5092 - keras_ssim: -1.1331e-02
Epoch 2/500
 - 0s - loss: 3.0928 - keras_psnr: 8.0095 - keras_ssim: 0.1026
Epoch 3/500
 - 0s - loss: 1.7876 - keras_psnr: 10.3904 - keras_ssim: 0.1972
Epoch 4/500
 - 0s - loss: 1.2391 - keras_psnr: 11.9820 - keras_ssim: 0.2463
Epoch 5/500
 - 0s - loss: 0.9861 - keras_psnr: 12.9737 - keras_ssim: 0.2930
Epoch 6/500
 - 0s - loss: 0.8232 - keras_psnr: 13.7578 - keras_ssim: 0.3409
Epoch 7/500
 - 0s - loss: 0.7001 - keras_psnr: 14.4613 - keras_ssim: 0.4129
Epoch 8/500
 - 0s - loss: 0.6171 - keras_psnr: 15.0094 - keras_ssim: 0.4681
Epoch 9/500
 - 0s - loss: 0.5537 - keras_psnr: 15.4801 - keras_ssim: 0.5136
Epoch 10/500
 - 0s - loss: 0.4928 - keras_psnr: 15.9861 - keras_ssim: 0.5405
Epoch 11/500
 - 0s - loss: 0.4456 - keras_psnr: 16.4231 - keras_ssim: 0.5552
Epoch 12/500
 - 0s - loss: 0.3984 - keras_psnr: 16.9094 - keras_ssim: 0.5704
Epoch 13/500
 - 0s - loss: 0.3484 

Epoch 104/500
 - 0s - loss: 0.0085 - keras_psnr: 33.6021 - keras_ssim: 0.9233
Epoch 105/500
 - 0s - loss: 0.0084 - keras_psnr: 33.6504 - keras_ssim: 0.9238
Epoch 106/500
 - 0s - loss: 0.0083 - keras_psnr: 33.6985 - keras_ssim: 0.9245
Epoch 107/500
 - 0s - loss: 0.0083 - keras_psnr: 33.7463 - keras_ssim: 0.9251
Epoch 108/500
 - 0s - loss: 0.0082 - keras_psnr: 33.7934 - keras_ssim: 0.9258
Epoch 109/500
 - 0s - loss: 0.0081 - keras_psnr: 33.8408 - keras_ssim: 0.9264
Epoch 110/500
 - 0s - loss: 0.0080 - keras_psnr: 33.8879 - keras_ssim: 0.9270
Epoch 111/500
 - 0s - loss: 0.0079 - keras_psnr: 33.9352 - keras_ssim: 0.9275
Epoch 112/500
 - 0s - loss: 0.0078 - keras_psnr: 33.9822 - keras_ssim: 0.9278
Epoch 113/500
 - 0s - loss: 0.0077 - keras_psnr: 34.0279 - keras_ssim: 0.9282
Epoch 114/500
 - 0s - loss: 0.0077 - keras_psnr: 34.0733 - keras_ssim: 0.9288
Epoch 115/500
 - 0s - loss: 0.0076 - keras_psnr: 34.1180 - keras_ssim: 0.9293
Epoch 116/500
 - 0s - loss: 0.0075 - keras_psnr: 34.1620 - keras

 - 0s - loss: 0.0038 - keras_psnr: 37.0760 - keras_ssim: 0.9588
Epoch 210/500
 - 0s - loss: 0.0038 - keras_psnr: 37.0992 - keras_ssim: 0.9589
Epoch 211/500
 - 0s - loss: 0.0038 - keras_psnr: 37.1226 - keras_ssim: 0.9591
Epoch 212/500
 - 0s - loss: 0.0038 - keras_psnr: 37.1452 - keras_ssim: 0.9593
Epoch 213/500
 - 0s - loss: 0.0038 - keras_psnr: 37.1670 - keras_ssim: 0.9595
Epoch 214/500
 - 0s - loss: 0.0037 - keras_psnr: 37.1883 - keras_ssim: 0.9596
Epoch 215/500
 - 0s - loss: 0.0037 - keras_psnr: 37.2097 - keras_ssim: 0.9598
Epoch 216/500
 - 0s - loss: 0.0037 - keras_psnr: 37.2315 - keras_ssim: 0.9600
Epoch 217/500
 - 0s - loss: 0.0037 - keras_psnr: 37.2536 - keras_ssim: 0.9602
Epoch 218/500
 - 0s - loss: 0.0037 - keras_psnr: 37.2757 - keras_ssim: 0.9603
Epoch 219/500
 - 0s - loss: 0.0036 - keras_psnr: 37.2976 - keras_ssim: 0.9605
Epoch 220/500
 - 0s - loss: 0.0036 - keras_psnr: 37.3190 - keras_ssim: 0.9607
Epoch 221/500
 - 0s - loss: 0.0036 - keras_psnr: 37.3400 - keras_ssim: 0.9608


Epoch 315/500
 - 0s - loss: 0.0025 - keras_psnr: 38.9783 - keras_ssim: 0.9716
Epoch 316/500
 - 0s - loss: 0.0025 - keras_psnr: 38.9934 - keras_ssim: 0.9718
Epoch 317/500
 - 0s - loss: 0.0025 - keras_psnr: 39.0081 - keras_ssim: 0.9718
Epoch 318/500
 - 0s - loss: 0.0024 - keras_psnr: 39.0224 - keras_ssim: 0.9720
Epoch 319/500
 - 0s - loss: 0.0024 - keras_psnr: 39.0359 - keras_ssim: 0.9719
Epoch 320/500
 - 0s - loss: 0.0024 - keras_psnr: 39.0484 - keras_ssim: 0.9721
Epoch 321/500
 - 0s - loss: 0.0024 - keras_psnr: 39.0595 - keras_ssim: 0.9721
Epoch 322/500
 - 0s - loss: 0.0024 - keras_psnr: 39.0688 - keras_ssim: 0.9723
Epoch 323/500
 - 0s - loss: 0.0024 - keras_psnr: 39.0763 - keras_ssim: 0.9722
Epoch 324/500
 - 0s - loss: 0.0024 - keras_psnr: 39.0820 - keras_ssim: 0.9725
Epoch 325/500
 - 0s - loss: 0.0024 - keras_psnr: 39.0881 - keras_ssim: 0.9723
Epoch 326/500
 - 0s - loss: 0.0024 - keras_psnr: 39.0970 - keras_ssim: 0.9726
Epoch 327/500
 - 0s - loss: 0.0024 - keras_psnr: 39.1114 - keras

 - 0s - loss: 0.0018 - keras_psnr: 40.3466 - keras_ssim: 0.9782
Epoch 421/500
 - 0s - loss: 0.0018 - keras_psnr: 40.3490 - keras_ssim: 0.9784
Epoch 422/500
 - 0s - loss: 0.0018 - keras_psnr: 40.3506 - keras_ssim: 0.9783
Epoch 423/500
 - 0s - loss: 0.0018 - keras_psnr: 40.3521 - keras_ssim: 0.9785
Epoch 424/500
 - 0s - loss: 0.0018 - keras_psnr: 40.3532 - keras_ssim: 0.9783
Epoch 425/500
 - 0s - loss: 0.0018 - keras_psnr: 40.3539 - keras_ssim: 0.9786
Epoch 426/500
 - 0s - loss: 0.0018 - keras_psnr: 40.3541 - keras_ssim: 0.9783
Epoch 427/500
 - 0s - loss: 0.0018 - keras_psnr: 40.3544 - keras_ssim: 0.9786
Epoch 428/500
 - 0s - loss: 0.0018 - keras_psnr: 40.3549 - keras_ssim: 0.9784
Epoch 429/500
 - 0s - loss: 0.0018 - keras_psnr: 40.3568 - keras_ssim: 0.9787
Epoch 430/500
 - 0s - loss: 0.0018 - keras_psnr: 40.3611 - keras_ssim: 0.9784
Epoch 431/500
 - 0s - loss: 0.0018 - keras_psnr: 40.3694 - keras_ssim: 0.9788
Epoch 432/500
 - 0s - loss: 0.0018 - keras_psnr: 40.3833 - keras_ssim: 0.9785


In [11]:
data[0].shape

(1, 488, 328, 3)